# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Antoni Climent Muñoz  <br>
Url: https://github.com/antoni-climent/Trabajo_algoritmos_optimizacion.git<br>
Problema:
> 1. Sesiones de doblaje <br>
> 2. ~~Organizar los horarios de partidos de La Liga~~<br>
> 3. ~~Combinar cifras y operaciones~~

Descripción del problema:(copiar enunciado)

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran toda la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

(*) La respuesta es obligatoria





                                        

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

Como tenemos 30 tomas a grabar y cada día se pueden completar 6, vamos a definir el problema como la creación del horario de 5 días.
Al utilizar una estrategia de fuerza bruta, miraremos todas las posibles soluciones que hay, guardando la mejor encontrada hasta el momento. Como tenemos 30 tomas y cada toma puede estar en cualquier día, si no tenemos en cuenta ninguna restricción vamos a tener 30! posibles permutaciones. 

No obstante, permutar tomas dentro del mismo día no afecta a la solución del problema. Teniendo en cuenta este hecho, vemos que cada toma estará relacionada con uno de los cinco valores distintos (uno por cada día), con lo que habrá 5<sup>30</sup> posibles soluciones.

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

En este problema se necesitan como mínimo dos estructuras de datos. Una que contenga la solución y otra con la asignación de actores para cada toma.
Para la primera, hemos utilizado un array de 30 elementos, uno por cada toma. Esta decisión de debe a la simplicidad de la representación y porque para ver las asignaciones de cada día solo hay que hacer un reshape del array a dimensión (5,6). Por otro lado, para guardar las restricciones del problema (la asignación toma-actor) hemos utilizado el mismo formato que se nos ha dado, una tabla. De esta forma es fácil de acceder a todos los actores que aparecen en una toma, pues basta con mirar la fila correspondiente.

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

La función objetivo está definida con el nombre "compute_cost", que toma una propuesta de horario y calcula el total de desplazamientos de los actores a lo largo de los 5 días. Como define el problema, esta variable es la que queremos minimizar, para abaratar los costes del doblaje.

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [3]:
import pandas as pd
import numpy as np
from itertools import permutations

In [4]:
# Function to compute the cost of a given schedule
def compute_cost(perm, data):
    perm = perm.reshape(5,6) # The schedule consists of 5 days and 6 sessions per day
    actor = np.zeros((5,10))
    for i, tomas in enumerate(perm):
        for toma in tomas:
            actor[i] = actor[i] + data[toma-1] # Counts the number of actors per day
    return sum([1 if a > 0 else 0 for a in actor.flatten()]) # Returns the cost of the schedule

In [6]:
# To implement the algorithm by brute force, we will do all possible permutations of the schedule.
# Notice that it can not be fully executed, as there are 30! permutations.
data = pd.read_csv('data.csv', index_col=0) # Load the data
array = np.array([i for i in range(1, 31)]) # Create an schedule example, so that we can get all the permutations
perms = permutations(array) # Iterable object


min_cost = 10000
for t, perm in enumerate(perms):
    c = compute_cost(np.array(perm), np.array(data))
    if c < min_cost:
        min_cost = c
    if t == 100000: # We put a counter to stop the loop after 100000 iterations, as otherwise the computer crashes.
        break
print(f'Min cost found: {min_cost}') # The minimum cost found

Min cost found: 36


Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

Nuestra implementación por fuerza bruta no tiene en cuenta ninguna restricción, simplemente va generando todas las posibles soluciones al problema y calcula su coste, guardando siempre el mínimo encontrado hasta el momento. Esto significa, como hemos visto en la pregunta sobre las posibilidades del problema, que la complejidad va a ser O(n!), siendo n el número de tomas a realizar. 


(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

In [15]:
# Genetic algorithm 
def genetic_algorithm(best_popu):
    data = pd.read_csv('data.csv', index_col=0) # Load problem data
    # Start the genetic algorithm
    for generation in range(15): # Number of generations
        # Generate new individuals
        new_popu = np.empty((0, 30))
        for i in range(len(best_popu)):
            for j in range(10):
                new_ind = best_popu[i].copy()
                ind1 = np.random.randint(0, 30)
                ind2 = np.random.randint(0, 30)
                new_ind[ind1], new_ind[ind2] = new_ind[ind2], new_ind[ind1]
                new_popu = np.vstack([new_popu, new_ind])

        # Compute cost for each individual
        costs = np.array([])
        for indiv in new_popu:
            costs = np.append(costs, compute_cost(indiv.astype(int), np.array(data)))

        # Select best individuals
        best = np.argsort(costs)[:1000]
        best_popu = [new_popu[i] for i in best]
        #print(f'Generaton {generation}: {min(costs)}') # Uncomment to see the cost of the best individual in each generation

    best = np.argsort(costs)[:1] # Select the best individual
    best_popu = [new_popu[i] for i in best]
    print(f'Best Population: {best_popu[0]}, with cost:  {costs[best][0]}')


Resolver este problema por fuerza bruta no es viable, así que el algoritmo genético concede una alternativa para solucionarlo. Veremos más adelante que el menor coste encontrado tras 15 generaciones es 27, lo cual mejora considerablemente la solución encontrada tras 100000 iteraciones del primer algoritmo.

(*)Calcula la complejidad del algoritmo

Respuesta

La complejidad del problema depende de la definición de tres hiperparámetros:
- El número de generaciones que se computan (g)
- El tamaño de la población que sobrevive (p)
- Y de los hijos que se generan a partir de cada superviviente (h)

En nuestra implementación se ha separado la computación de la función objetivo del bucle que genera los nuevos individuos, así que hay dos bucles con el mismo número de iteraciones, dejándonos con: O(g*(2*(p\*h))) que finalmente queda como O(g\*p*h)

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [17]:
# Podemos consider que los datos de entrada aleatorios sea la población inicial:
array = np.array([i for i in range(1, 31)]) # Create array of assignments toma - dia
best_popu = [np.random.permutation(array) for i in range(1000)] # Generate initial population

Aplica el algoritmo al juego de datos generado

Respuesta

In [16]:
for _ in range(3): # Generate 3 different initial populations and run the genetic algorithm
    array = np.array([i for i in range(1, 31)]) # Create array of assignments toma - dia
    best_popu = [np.random.permutation(array) for i in range(1000)] # Generate initial population
    genetic_algorithm(best_popu) # Run the genetic algorithm

Best Population: [25.  5.  4.  9. 30. 16. 11.  8. 21. 10. 29. 26.  3. 15.  6. 28. 13.  7.
  1. 22. 20. 12.  2. 27. 18. 24. 19. 17. 14. 23.], with cost:  27.0
Best Population: [27.  9. 25.  6. 16.  7. 24. 19. 14. 23. 21. 18.  2. 13. 30. 20. 17. 28.
  5. 15.  4. 22.  3. 11. 26. 12.  1. 10. 29.  8.], with cost:  27.0
Best Population: [29.  8. 11. 21. 10. 26. 28.  9. 27. 25. 16.  6. 18. 17. 14. 23. 24. 19.
  7.  2. 12. 22.  1. 20. 30.  3.  4. 13. 15.  5.], with cost:  27.0


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

[1] Manual de la asignatura, *Raúl Reyero Díez* <br>

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta

Como trabajo futuro, se podría considerar no coger solo a los mejores individuos de la población, introduciendo una probabilidad de que se escojan soluciones peores (como en GRASP), de manera que aumentara la búsqueda en amplitud.

Durante la etapa inicial del proyecto se consideró aplicar un algoritmo voraz, que construyera la solución mirando los actores ya asignados, de manera que se maximizara el número de tomas que hace un actor durante un día. Sería muy interesante ver si una solución voraz puede aportar una mejor solución para este problema. 

Por otro lado, durante la fase de diseño del algoritmo se impuso la restricción de que las tomas se realizaran a lo largo de 5 días, ya que cuadra bien teniendo en cuenta que hay 30 tomas a grabar y cada día se graban 6 como máximo. No obstante, nada garantiza que no se puedan encontrar soluciones mejores si se considera realizar las tomas en más días, así que sería un buen factor a explorar.

Por último, se me ocurre una variación del problema en el que hay actores que se llevan mal y exigen cobrar más por trabajar juntos, de manera que haya una sola variable a optimizar, pero que dependa de varios factores.